In [2]:
library(lubridate)

# Utilities for creation and manipulation of Interval objects

```r
interval(start = NULL, end = NULL, tzone = tz(start))

start %--% end

is.interval(x)

int_start(int)

int_start(int) <- value

int_end(int)

int_end(int) <- value

int_length(int)

int_flip(int)

int_shift(int, by)

int_overlaps(int1, int2)

int_standardize(int)

int_aligns(int1, int2)

int_diff(times)
```

### Examples

**`interval()`** creates an Interval object with the specified start and end dates. If the start date occurs before the end date, the interval will be positive. Otherwise, it will be negative. Character vectors in ISO 8601 format are supported from v1.7.2.

In [11]:
# Positive
interval(ymd(200101), ymd(200108))

# equivalent
ymd(200101) %--% ymd(200108)

[1] 2020-01-01 UTC--2020-01-08 UTC

[1] 2020-01-01 UTC--2020-01-08 UTC

In [12]:
# Negative
interval(ymd(200108), ymd(200101))

# equivalent
ymd(200108) %--% ymd(200101)

[1] 2020-01-08 UTC--2020-01-01 UTC

[1] 2020-01-08 UTC--2020-01-01 UTC

In [8]:
date1 <- ymd_hms("2009-03-08 01:59:59")
date2 <- ymd_hms("2000-02-29 12:00:00")

interval(date2, date1)

[1] 2000-02-29 12:00:00 UTC--2009-03-08 01:59:59 UTC

In [9]:
### ISO Intervals

interval("2007-03-01T13:00:00Z/2008-05-11T15:30:00Z")

interval("2007-03-01T13:00:00Z/P1Y2M10DT2H30M")

interval("P1Y2M10DT2H30M/2008-05-11T15:30:00Z")

interval("2008-05-11/P2H30M")

[1] 2007-03-01 13:00:00 UTC--2008-05-11 15:30:00 UTC

[1] 2007-03-01 13:00:00 UTC--2008-05-11 15:30:00 UTC

[1] 2007-03-01 13:00:00 UTC--2008-05-11 15:30:00 UTC

[1] 2008-05-11 UTC--2010-11-11 02:00:00 UTC

In [10]:
### More permisive parsing (as long as there are no intermittent / characters)
interval("2008 05 11/P2hours 30minutes")

interval("08 05 11/P 2h 30m")


[1] 2008-05-11 UTC--2008-05-11 02:30:00 UTC

[1] 2008-05-11 UTC--2010-11-11 02:00:00 UTC

<hr>

is x an interval object?

In [13]:
is.interval(period(1, 'month'))

[1] FALSE

In [14]:
is.interval(ymd(101010) %--% ymd(101020))

[1] TRUE

<hr>

`int_start()`/`int_end()` and `int_start<-()`/`int_end<-()` are "accessors" and "setters" respectively of the start/end date of an interval.

In [15]:
event <- ymd(201121) %--% ymd(201125)

event

[1] 2020-11-21 UTC--2020-11-25 UTC

In [16]:
# get start date of interval
int_start(event)

[1] "2020-11-21 UTC"

In [17]:
# get end date of interval
int_end(event)

[1] "2020-11-25 UTC"

In [18]:
# set start date to 2020/11/20

int_start(event) <- ymd(201120)

event

[1] 2020-11-20 UTC--2020-11-25 UTC

<hr>

**`int_flip()`** reverses the order of the start date and end date in an interval. The new interval takes place during the same timespan as the original interval, but has the opposite direction.


In [21]:
event

int_start(event)

int_end(event)

[1] 2020-11-20 UTC--2020-11-25 UTC

[1] "2020-11-20 07:00:00 +07"

[1] "2020-11-25 07:00:00 +07"

In [23]:
event_flipped <- int_flip(event)

event_flipped

int_start(event_flipped)

int_end(event_flipped)

[1] 2020-11-25 UTC--2020-11-20 UTC

[1] "2020-11-25 07:00:00 +07"

[1] "2020-11-20 07:00:00 +07"

<hr>

**`int_shift()`** shifts the start and end dates of an interval up or down the timeline by a specified amount. Note that this may change the exact length of the interval if the interval is shifted by a Period object. Intervals shifted by a Duration or difftime object will retain their exact length in seconds.

In [29]:
int <- interval(ymd("2001-01-01"), ymd("2002-01-01"))

# return a new Interval
int_shift(int, duration(days = 11))

[1] 2001-01-12 UTC--2002-01-12 UTC

In [30]:
int_shift(int, duration(-1, 'hour'))

[1] 2000-12-31 23:00:00 UTC--2001-12-31 23:00:00 UTC

<hr>

**`int_length()`**: return the length of an Interval in seconds

In [24]:
event

[1] 2020-11-20 UTC--2020-11-25 UTC

In [26]:
int_length(event)

int_length(event_flipped)

[1] 432000

[1] -432000

<hr>

**`int_overlaps()`** tests if two intervals overlap.

In [31]:
int1 <- interval(ymd("2001-01-01"), ymd("2002-01-01"))
int2 <- interval(ymd("2001-06-01"), ymd("2002-06-01"))
int3 <- interval(ymd("2003-01-01"), ymd("2004-01-01"))

In [33]:
int_overlaps(int1, int2)

[1] TRUE

In [34]:
int_overlaps(int1, int3)

[1] FALSE

<hr> 

**`int_standardize()`** ensures all intervals in an interval object are positive. If an interval is not positive, flip it so that it retains its endpoints but becomes positive.

In [36]:
event   # positive interval

# nothing change
int_standardize(event)

[1] 2020-11-20 UTC--2020-11-25 UTC

[1] 2020-11-20 UTC--2020-11-25 UTC

In [37]:
event_flipped # negative interval

# change negative interval to positive

int_standardize(event_flipped)

[1] 2020-11-25 UTC--2020-11-20 UTC

[1] 2020-11-20 UTC--2020-11-25 UTC

<hr>

**`int_aligns()`** tests if two intervals share an endpoint. The direction of each interval is ignored. `int_align` tests whether the earliest or latest moments of each interval occur at the same time.

In [39]:
int1 <- interval(ymd("2001-01-01"), ymd("2002-01-01"))
int2 <- interval(ymd("2001-06-01"), ymd("2002-01-01"))
int3 <- interval(ymd("2003-01-01"), ymd("2004-01-01"))


In [41]:
int_aligns(int1, int2)

int_aligns(int1, int3)

[1] TRUE

[1] FALSE

<hr>

**`int_diff()`** returns the intervals that occur between the elements of a vector of date-times. `int_diff()` is similar to the POSIXt and Date methods of `diff()`, but returns an Interval object instead of a difftime object.

In [42]:
values <- now() + days(1:10)

values

 [1] "2020-11-14 17:24:56 +07" "2020-11-15 17:24:56 +07"
 [3] "2020-11-16 17:24:56 +07" "2020-11-17 17:24:56 +07"
 [5] "2020-11-18 17:24:56 +07" "2020-11-19 17:24:56 +07"
 [7] "2020-11-20 17:24:56 +07" "2020-11-21 17:24:56 +07"
 [9] "2020-11-22 17:24:56 +07" "2020-11-23 17:24:56 +07"

In [43]:
int_diff(values)

[1] 2020-11-14 17:24:56 +07--2020-11-15 17:24:56 +07
[2] 2020-11-15 17:24:56 +07--2020-11-16 17:24:56 +07
[3] 2020-11-16 17:24:56 +07--2020-11-17 17:24:56 +07
[4] 2020-11-17 17:24:56 +07--2020-11-18 17:24:56 +07
[5] 2020-11-18 17:24:56 +07--2020-11-19 17:24:56 +07
[6] 2020-11-19 17:24:56 +07--2020-11-20 17:24:56 +07
[7] 2020-11-20 17:24:56 +07--2020-11-21 17:24:56 +07
[8] 2020-11-21 17:24:56 +07--2020-11-22 17:24:56 +07
[9] 2020-11-22 17:24:56 +07--2020-11-23 17:24:56 +07

# Change an object to an Interval

**`as.interval`** changes difftime, Duration, Period and numeric class objects to intervals that begin at the specified date-time. Numeric objects are first coerced to timespans equal to the numeric value in seconds.

```r
as.interval(x, start, ...)
```

### Examples

In [3]:
diff <- make_difftime(days = 31) #difftime

as.interval(diff, ymd("2009-01-01"))

as.interval(diff, ymd("2009-02-01"))


[1] 2009-01-01 UTC--2009-02-01 UTC

[1] 2009-02-01 UTC--2009-03-04 UTC

In [4]:
dur <- duration(days = 31) #duration
as.interval(dur, ymd("2009-01-01"))

as.interval(dur, ymd("2009-02-01"))


[1] 2009-01-01 UTC--2009-02-01 UTC

[1] 2009-02-01 UTC--2009-03-04 UTC

In [5]:
per <- period(months = 1) #period
as.interval(per, ymd("2009-01-01"))

as.interval(per, ymd("2009-02-01"))

[1] 2009-01-01 UTC--2009-02-01 UTC

[1] 2009-02-01 UTC--2009-03-01 UTC

In [7]:
as.interval(3600, ymd("2009-01-01"))

[1] 2009-01-01 UTC--2018-11-10 UTC

# Does a date (or interval) fall within an interval?

Check whether `a` lies within the interval `b`, inclusive of the endpoints.
```r
a %within% b
```

**Arguments**  
`a`	
An interval or date-time object.

`b`	
Either an interval vector, or a list of intervals.

If b is an interval it is recycled to the same length as a. If b is a list of intervals, a is checked if it falls within any of the intervals, i.e. a `%within% list(int1, int2)` is equivalent to a `%within% int1 | a %within% int2`.

### Examples

In [8]:
int <- interval(ymd("2001-01-01"), ymd("2002-01-01"))

int2 <- interval(ymd("2001-06-01"), ymd("2002-01-01"))

In [9]:
ymd(20010606) %within% int

[1] TRUE

In [11]:
ymd("1999-01-01") %within% int

[1] FALSE

In [10]:
int2 %within% int

[1] TRUE

In [13]:
## recycling
dates <- ymd(c("2014-12-20", "2014-12-30", "2015-01-01", "2015-01-03"))

blackouts<- c(interval(ymd("2014-12-30"), ymd("2014-12-31")),
              interval(ymd("2014-12-30"), ymd("2015-01-03")))
dates %within% blackouts

[1] FALSE  TRUE FALSE  TRUE

In [15]:
## within ANY of the intervals of a list
dates <- ymd(c("2014-12-20", "2014-12-30", "2015-01-01", "2015-01-03"))
blackouts<- list(interval(ymd("2014-12-30"), ymd("2014-12-31")),
                 interval(ymd("2014-12-30"), ymd("2015-01-03")))

dates %within% blackouts

[1] FALSE  TRUE  TRUE  TRUE